<a href="https://colab.research.google.com/github/Francimaria/Machine-Learning-Python/blob/master/part2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import pandas as pd
import numpy as np

from scipy.stats import multivariate_normal
import math

from sklearn import preprocessing
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [102]:
#dataset
view1 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-fac.csv', header=None, delim_whitespace=True)
view2 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-fou.csv', header=None, delim_whitespace=True)
view3 = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/mfeat-kar.csv', header=None, delim_whitespace=True)

#normalização
# Default=l2-norm (é o comprimento euclidiano usual, ou seja, a raiz quadrada da soma dos elementos do vetor ao quadrado.)
view1 = preprocessing.normalize(view1)
view2 = preprocessing.normalize(view2)
view3 = preprocessing.normalize(view3) 
"""n = 2000
view1 = view1[:n,[0,1,2,3,4]]
view2 = view2[:n,[0,1,2,3,4]]
view3 = view3[:n,[0,1,2,3,4]]"""

classes = pd.read_csv('https://raw.githubusercontent.com/Francimaria/Machine-Learning-Python/master/cluster_membership_tres_m.csv', header=None, delim_whitespace=True)

#classes = classes[:n]

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=30)
for train_index, test_index in rkf.split(view1,y):
  v1_train, v1_test = view1[train_index], view1[test_index]
  v2_train, v2_test = view2[train_index], view2[test_index]
  v3_train, v3_test = view3[train_index], view3[test_index]
  y_train, y_test = y[train_index], y[test_index]

X_train = [v1_train,v2_train,v3_train]
X_test = [v1_test,v2_test,v3_test]

bg.fit(X_train,y_train)

y_pred = bg.predict(X_test)
a = accuracy_score(y_test, y_pred)
print("acuracy ",a)


{0: 271, 1: 54, 2: 22, 3: 243, 4: 101, 5: 101, 6: 279, 7: 196, 8: 192, 9: 341}
{0: 271, 1: 54, 2: 22, 3: 243, 4: 101, 5: 101, 6: 279, 7: 196, 8: 192, 9: 341}
{0: 271, 1: 54, 2: 22, 3: 243, 4: 101, 5: 101, 6: 279, 7: 196, 8: 192, 9: 341}


/usr/local/lib/python3.6/dist-packages/scipy/stats/_multivariate.py:522: RuntimeWarning: overflow encountered in exp
  out = np.exp(self._logpdf(x, mean, psd.U, psd.log_pdet, psd.rank))


OverflowError: ignored

In [103]:
#Based on: https://towardsdatascience.com/how-to-impliment-a-gaussian-naive-bayes-classifier-in-python-from-scratch-11e0b80faf5a
class gaussClf:
    def separate_by_classes(self, X, y):
        ''' This function separates our dataset in subdatasets by classes '''
        self.classes = np.unique(y)
        classes_index = {}
        subdatasets = {}
        cls, counts = np.unique(y, return_counts=True)
        self.class_freq = dict(zip(cls, counts))
        print(self.class_freq)
        for class_type in self.classes:
            classes_index[class_type] = np.argwhere(y==class_type)
            subdatasets[class_type] = X[classes_index[class_type], :]
            self.class_freq[class_type] = self.class_freq[class_type]/sum(list(self.class_freq.values()))
        return subdatasets
        
    def meanCov(self,X,y):
      separated_X = self.separate_by_classes(X, y)
      mean_c = {}
      sigma_c = {}
      for class_type in self.classes:
          # Aqui calculamos a média e a matriz Sigma diagonal
          Xc = np.asmatrix(separated_X[class_type])
          mean_c[class_type] = np.asarray(np.mean(Xc, axis=0))[0]
          cov_c = np.zeros((Xc.shape[1], Xc.shape[1]))
          for j in range(Xc.shape[1]):
            var = np.var(Xc[:,[j]].reshape((1,Xc.shape[0])))
            cov_c[j][j] = var          
          sigma_c[class_type] = cov_c
      return mean_c, sigma_c

    def fit(self, X, y):
        ''' The fitting function '''
        self.meansView1, self.sigmaView1 = self.meanCov(X[0],y)
        self.meansView2, self.sigmaView2 = self.meanCov(X[1],y)
        self.meansView3, self.sigmaView3 = self.meanCov(X[2],y)

    def calculate_probability(self, x, mean, sigmad):     
      normal_mult = multivariate_normal.pdf(x,mean=mean,cov=sigmad)
      return normal_mult

    #Precisa de ajuste
    def predict_proba(self, X):
      #número de view
        L = len(X)
        view1 = X[0]
        view2 = X[1]
        view3 = X[2]
        #colunas
        ''' Prediz a probabilidade para todas as classes log-verossimilhança'''
        self.class_prob = {cls:((L-1)*math.log(self.class_freq[cls], math.e)) for cls in self.classes}
        
        for cls in self.classes:  
            if (self.calculate_probability(view1, self.meansView1[cls], self.sigmaView1[cls]) < 0):
              print("classe probabili - ", self.class_prob)     
            pView1 = (math.log(self.calculate_probability(view1, self.meansView1[cls], self.sigmaView1[cls]), math.e))
            pView2 = (math.log(self.calculate_probability(view2, self.meansView2[cls], self.sigmaView2[cls]), math.e))
            pView3 = (math.log(self.calculate_probability(view3, self.meansView3[cls], self.sigmaView3[cls]), math.e))
            self.class_prob[cls]+=(pView1 + pView2 + pView3)
        self.class_prob = {cls: math.e**self.class_prob[cls] for cls in self.class_prob}
        return self.class_prob

    def predict(self, X):
        ''' Essa função prediz a probabilidade da classe de uma amostra '''
        pred = []
        v1 = X[0]
        v2 = X[1]
        v3 = X[2]
        for i in range(len(X[0])):
            pred_class = None
            max_prob = 0
            x =[v1[i],v2[i],v3[i]]
            for cls, prob in self.predict_proba(x).items():
                if prob>max_prob:
                    max_prob = prob
                    pred_class = cls
            pred.append(pred_class)
        return pred

In [110]:
# Bayes Teste
view1 = np.random.randint(10, size=(10,50))
view2 = np.random.randint(10, size=(10,50))
view3 = np.random.randint(10, size=(10,50))

view1 = preprocessing.normalize(view1)
view2 = preprocessing.normalize(view2)
view3 = preprocessing.normalize(view3)

X = [view1,view2,view3] 
y = [2,1,1,0,0,1,1,2,0,2]
#X_train, X_test, y_train, y_test = train_test_split(X, y)
bg = gaussClf()
bg.fit(X,y)
print(X)
y_pred = bg.predict(X)
print(y_pred)
a = accuracy_score(y, y_pred)
print("acuracy ",a)






{0: 3, 1: 4, 2: 3}
{0: 3, 1: 4, 2: 3}
{0: 3, 1: 4, 2: 3}
[array([[0.26233382, 0.        , 0.08744461, 0.26233382, 0.0582964 ,
        0.17488921, 0.0291482 , 0.17488921, 0.0291482 , 0.20403741,
        0.17488921, 0.23318561, 0.08744461, 0.08744461, 0.11659281,
        0.17488921, 0.08744461, 0.14574101, 0.0291482 , 0.20403741,
        0.11659281, 0.23318561, 0.17488921, 0.11659281, 0.17488921,
        0.08744461, 0.20403741, 0.23318561, 0.23318561, 0.14574101,
        0.14574101, 0.0582964 , 0.0582964 , 0.0582964 , 0.0291482 ,
        0.20403741, 0.11659281, 0.0291482 , 0.0291482 , 0.        ,
        0.11659281, 0.0291482 , 0.17488921, 0.0291482 , 0.11659281,
        0.0291482 , 0.0291482 , 0.20403741, 0.0582964 , 0.11659281],
       [0.        , 0.0285249 , 0.        , 0.22819916, 0.11409958,
        0.14262448, 0.05704979, 0.        , 0.25672406, 0.05704979,
        0.11409958, 0.11409958, 0.        , 0.19967427, 0.11409958,
        0.        , 0.19967427, 0.14262448, 0.05704979, 0

ValueError: ignored